In [62]:
import pandas as pd
import numpy as np
import os
import pkg_resources
import nltk
from symspellpy.symspellpy import SymSpell
from symspellpy.symspellpy import SymSpell, Verbosity  # import the module


def load_dictionnary(dict_path):
    with open(dict_path) as f :
        L = f.readlines()
    words = []
    for i in L:
        words.append(i.split(" ")[0])
    return words


def load_lexique(lexique_path):
    with open(lexique_path) as f:
        L = f.readlines()
    words = []
    for i in L:
        words.append(i.split("\t")[0])
    return words

def spell_correction(texte):
    input_term = texte
    # max edit distance per lookup (per single word, not per whole input string)
    max_edit_distance_lookup = 2
    suggestions = sym_spell.lookup_compound(input_term,
                                            max_edit_distance_lookup)
    # display suggestion term, edit distance, and term frequency
    for suggestion in suggestions:
        print("{}, {}, {}".format(suggestion.term, suggestion.distance,
                                  suggestion.count))
    if(len(suggestions)>0):
        return suggestions[0].term
    else:
        print("error with : ",texte)
        return texte

#print(spell_correction("bonjour"))





In [5]:
import unicodedata

from bs4 import BeautifulSoup
import sqlite3
import requests
import re
import json
import pandas as pd

class scrapper:
    @staticmethod
    def get_arks(year):
        url_base = "https://gallica.bnf.fr/ark:/12148/cb34363188x/date"+str(year)+"0101"
        req = requests.get(url_base, headers={'User-Agent': 'Mozilla/5.0'})
        p = req.content
        L = re.findall(r"https://gallica.bnf.fr/ark:/(\d*/(bpt\w*))?",str(p))
        S = set()
        for i in L:
            if(i[1] != ""):
                S.add(i[1])
        return S
    @staticmethod
    def get_pagination(ark):
        """fonction qui renvoie le nombre de pages d'un documents"""
        #print(cpt,"--> https://gallica.bnf.fr/"+i+"/f7/highres")
        req = requests.get("https://gallica.bnf.fr/services/Pagination?ark=" + ark)
        p = req.content
        pages = re.search(r"<nbVueImages>(\d*)</nbVueImages>", str(p))
        return pages.groups()[0]
    @staticmethod
    def get_page(ark, page, number = 1):
        
        req = requests.get(f"https://gallica.bnf.fr/ark:/12148/{ark}/f{page}n{number}.texteBrut")
        return req.content
    @staticmethod
    def get_document(ark, mode = "texteBrut"):
        req = requests.get("https://gallica.bnf.fr/ark:/12148/"+ark+"."+ mode)
        return req.content
    

In [118]:
class Cleaner:
    def __init__(self, directory):
        self.directory = directory
        self.dict_path = "../ressources/fr-100k.txt"
        self.stopwords = list(nltk.corpus.stopwords.words('french'))
        self.lexique_path = "../ressources/Lexique383.tsv"
        self.words = load_dictionnary() + self.stopwords + load_lexique()
        self.corrected = {}
        
        self.max_edit_distance_dictionary = 2
        self.prefix_length = 7
        self.sym_spell = SymSpell(self.max_edit_distance_dictionary, self.prefix_length)
        self.dictionary_path = "../ressources/fr-100k.txt"
        self.sym_spell.load_dictionary(self.dictionary_path, term_index=0, count_index=1)
        pass
    def extract(self, file):
        soup = BeautifulSoup(file , "html.parser")
        df = pd.DataFrame(columns=["page", "arrêt", "date", "juridiction"])
        Decision, notes, page, new_page, new_decision, count = False, False, 0, True, False,1
        for tag in soup.body :
            if count == 15:
                count = 0
                Decision = False
            new_decision = False
            string = tag.get_text()
            if tag.name == "hr":
                page += 1
                notes = False
                new_page  = True
            if tag.name == "p" and string is not None and not new_page:
                m1 = re.match(r"(^.*?(La Cour,|L(A|À|a) COUR)(?! DE)(.+)$|^JUGEMENT\.?\s?$|^A\s?R\s?R\s?(Ê|E)\s?T\.\s?$)", string)
                m2 = re.match(r"(.*?)D(u|û|ù)(.+?)(—|–|-|–|–)(.+)", string ) 
                
                if not Decision and m1:
                    Decision = True
                    text = ""
                    count = 1
                    if m1.groups()[3] != None:
                        text = str(m1.groups()[3])
                    First_page = page
                    new_decision = True
                if Decision and m2:
                    if count < 15:
                        if(new_decision):
                            text =m2.groups()[0]
                        else:
                            text += m2.groups()[0]
                        date = m2.groups()[2]
                        juridiction = m2.groups()[4]
                        df = df.append({'page' : First_page, 'arrêt' : text , "date": date, "juridiction" : juridiction},ignore_index=True)
                    Decision = False
                    text = ''
                elif not notes and Decision and not new_decision:
                    if not re.match(r"^\(\d*\).+$", string):
                        count +=1
                        text+= string + "\n"
                    else:
                        notes = True
                else:
                    pass
            else :
                new_page = False
        return df
    def save(self, df, ark):
        df.to_csv(f"{self.directory}/{ark}.csv", encoding="utf-8")
        pass
    def postProcess(self, df):
        # fix mix date-juridiction
        Rows_contains_ = df['date'].str.contains(r"(—|–|-)")
        for i, row in df[Rows_contains_].iterrows():
            m = re.search(r"(.+?)(—|–|-|—)(.+)(—|–|-|—)?.*", row["date"] )
            if m:
                df.at[i, "date"] = m.groups()[0]
                df.at[i, "juridiction"] = m.groups()[2]
        #if still not fixed --> drop them
        Rows_contains_ = df['date'].str.contains(r"(—|–|-)")
        df = df[Rows_contains_ == False]
        # drop date too long
        leng = df["date"].str.len()
        df = df[leng < 25] # drop too long date
        # drop date with no number
        number = df["date"].str.contains("^\D*$")
        df = df[number== False] # drop too long date
        length_decision = df.arrêt.str.len()
        # drop decision too short
        df = df[length_decision > 100]
        # drop juridiction too long
        for i , row in df.iterrows():
            m = re.search(r"(.+?)(—|–|-|—|,|;).*", row["juridiction"] )
            if m:
                df.at[i, "juridiction"] = m.groups()[0]
        
        # process the juridiction
        
        return df  
    def spell_check(self, df):
        df["arrêt"] = df["arrêt"].apply(self.correct)
        return df
    def correct(self,text):
        ntokens= []
        tokens = re.split('\s|,|\.|;|—|–|-|–|–|\n|:|\!|\?',text)
        for t in tokens:
            if(str(t).lower().isalpha() and not str(t).lower() in self.words and not str(t)[0].isupper()):
                if str(t) in self.corrected:
                    nt = self.corrected[t]
                else:
                    nt = t
                    suggestion = self.sym_spell.lookup_compound(t, 2)
                    if len(suggestion)> 0 : 
                        nt = suggestion[0].term
                    
                    self.corrected[t] = nt
                print(t + " -> " + nt)
                ntokens.append(nt)

            else:
                ntokens.append(t)
        return " ".join(ntokens)
                
  

In [126]:
years = scrapper.get_arks(1817)
years

{'bpt6k5791611b', 'bpt6k5791626s'}

In [112]:
test = scrapper.get_page("bpt6k5791626s", 1, 100 )

In [119]:
c = Cleaner("test")
#file = scrapper.get_document("bpt6k5548364r")


In [120]:
c = Cleaner("test")
df_test = c.extract(test)
df_test

,page,arrêt,date,juridiction
0,6,", -Faisant droit sur l'appel interjeté par Bi...",n mars 1816.,Cour royale de Paris. - Prés. M. Séguier - Co...
1,8,"LA COUR, - Considérant qu'au 15 ventôse an 6, ...",6,avril 1816.- Cour royale de Paris.
2,9,"LA COUR , - Statuant sur le renvoi ordonné par...",18 mars 1816.,Cour royale de Paris. - Plaid. MM. Gauthier e...
3,10,"LA COUR , - Considérant que d'après l'art. 559...",n décembre 1815.,Cour de Colmar.
4,14,"LA COUR, - Adoptant les motifs des premiers ju...",II janvier 1816.,Cour royale de Paris. - V. chambre.-Présid. M...
5,15,"» LA COUR, - Joint l'appel principal interjeté...",11 mai 1816.,Cour royale de Paris.
6,20,"LA COUR, - Attendu que le délai de trois jours...",21 mars 1815.,Cour royale d'Angers.
7,45,"LA COUR, - Attendu que la fin de non-recevoir ...",22 avril 1815.,Cour royale de Colmar.
8,47,"LA COUR, - Considérant que l'adjudication du d...",16 mars 1816,Cour royale de Paris. - 3e. chambre Prés. M. ...
9,48,"LA COUR, - Adoptant les motifs des premiers ju...",2 mai 1816.,Cour royale de Paris.


In [121]:
df_test = c.postProcess(df_test)

/home/chekirou/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [122]:
df_test

,page,arrêt,date,juridiction
0,6,", -Faisant droit sur l'appel interjeté par Bi...",n mars 1816.,Cour royale de Paris.
1,8,"LA COUR, - Considérant qu'au 15 ventôse an 6, ...",6,avril 1816.
2,9,"LA COUR , - Statuant sur le renvoi ordonné par...",18 mars 1816.,Cour royale de Paris.
3,10,"LA COUR , - Considérant que d'après l'art. 559...",n décembre 1815.,Cour de Colmar.
4,14,"LA COUR, - Adoptant les motifs des premiers ju...",II janvier 1816.,Cour royale de Paris.
5,15,"» LA COUR, - Joint l'appel principal interjeté...",11 mai 1816.,Cour royale de Paris.
6,20,"LA COUR, - Attendu que le délai de trois jours...",21 mars 1815.,Cour royale d'Angers.
7,45,"LA COUR, - Attendu que la fin de non-recevoir ...",22 avril 1815.,Cour royale de Colmar.
8,47,"LA COUR, - Considérant que l'adjudication du d...",16 mars 1816,Cour royale de Paris.
9,48,"LA COUR, - Adoptant les motifs des premiers ju...",2 mai 1816.,Cour royale de Paris.


In [123]:
df_test = c.spell_check(df_test)

interjeté -> interjeta
afinde -> afin de
ar -> par
rérages -> ré rages
opposans -> opposant
interjeté -> interjeta
jugemens -> jugement
placités -> pl cités
changemens -> changement
placités -> pl cités
interjeté -> interjeta
interjeté -> interjeta
assujéties -> assuré les
appelans -> appelons
instrumenlaires -> instrumentaires
rejelée -> rejette
originaiie -> originaire
demaine -> domaine
privilége -> privilège
interjeté -> interjeta
enfans -> enfants
enfans -> enfants
enfans -> enfants
resulter -> résulter
détruires -> détruire
éruendant -> rendant
interjeté -> interjeta
erremens -> errements
ordonneque -> ordonne que
interjeté -> interjeta
interjeté -> interjeta
émendant -> étendant
interjeté -> interjeta
portéeau -> portée au
émendant -> étendant
dévoilassent -> dévoila sent
émendant -> étendant
tément -> dément
contestans -> contestant
émendant -> étendant
émandant -> émanant
résuite -> résulte
opposans -> opposant
interjeté -> interjeta
émendant -> étendant
annulle -> annuelle
in

In [124]:
df_test

,page,arrêt,date,juridiction
0,6,Faisant droit sur l'appel interjeta par Bi...,n mars 1816.,Cour royale de Paris.
1,8,LA COUR Considérant qu'au 15 ventôse an 6 ...,6,avril 1816.
2,9,LA COUR Statuant sur le renvoi ordonné par...,18 mars 1816.,Cour royale de Paris.
3,10,LA COUR Considérant que d'après l'art 559...,n décembre 1815.,Cour de Colmar.
4,14,LA COUR Adoptant les motifs des premiers ju...,II janvier 1816.,Cour royale de Paris.
5,15,» LA COUR Joint l'appel principal interjeta...,11 mai 1816.,Cour royale de Paris.
6,20,LA COUR Attendu que le délai de trois jours...,21 mars 1815.,Cour royale d'Angers.
7,45,LA COUR Attendu que la fin de non recevoir ...,22 avril 1815.,Cour royale de Colmar.
8,47,LA COUR Considérant que l'adjudication du d...,16 mars 1816,Cour royale de Paris.
9,48,LA COUR Adoptant les motifs des premiers ju...,2 mai 1816.,Cour royale de Paris.


In [125]:
df_test.loc[34].arrêt

"LA COUR reçoit François Charles Decambray  opposant à l'exécution de l'arrêt par défaut  du 13 juin dernier  faisant droit sur l'appel interjeta par ledit Decambray  du jugement rendu par le tribunal civil de Rheirns  le 1er  février 1816   Considérant  sur la restriction de l'hypothèque légale de la femme Decambray  à certains immeubles   stipulée en l'article 12 de leur contrat de mariage  que le mineur habile à contracter mariage  quand il est assisté  dans le contrat  des personnes dont le consentement est nécessaire pour la validité du mariage  peut y stipuler et consentir toutes les conventions permises au majeur  telles que donation et restriction de l'hypothèque lé(1)  lé(1) paraît que la dame Decambray  sans attendre l'événement de sa demande   avait par précaution pris des inscriptions sur les biens de sou mari autres que ceux affectés par le contrat de mariage   gale    Que la renonciation à toute hypothèque est interdite au mineur  comme au majeur  mais la clause de réduct

In [97]:
df_test["arrêt"] = df_test["arrêt"].apply(c.spell_check)

interjeta, 1, 8635
interjeté -> interjeta
afin de, 1, 67800
afinde -> afin de
par, 1, 634490311
ar -> par
ré rages, 1, 0
rérages -> ré rages
opposant, 1, 540147
opposans -> opposant
interjeté -> interjeta
jugement, 1, 9670279
jugemens -> jugement
pl cités, 1, 3
placités -> pl cités
changement, 1, 6908588
changemens -> changement
placités -> pl cités
interjeté -> interjeta
interjeté -> interjeta
assuré les, 3, 5695
assujéties -> assuré les
appelons, 1, 800698
appelans -> appelons
instrumentaires, 1, 8673
instrumenlaires -> instrumentaires
rejette, 2, 911178
rejelée -> rejette
originaire, 1, 1221137
originaiie -> originaire
domaine, 1, 12582308
demaine -> domaine
privilège, 1, 2194637
privilége -> privilège


In [98]:
df_test.loc[8].arrêt

"LA COUR    Considérant que l'adjudication du demaine de Sennevoy  faite à Méat le 16 mai 1812   a été transcrite le 29 du même mois   que  lors de cette transcription  il n'existait  de même qu'il n'est survenu  dans le délai de quinzaine qui l'a suivie  aucuns inscription de la part de Richardot et consorts  pour conserver   soit leur privilége  soit une hypothèque quelconque   met l'appellation au néant   ordonne que ce dont est appel sortira effet   condamne les appelai » en l'amende et aux dépens   "

In [87]:
c.corrected

{'interjeté': 'interjeta',
 'afinde': 'afin de',
 'ar': 'par',
 'rérages': 'ré rages',
 'opposans': 'opposant',
 'jugemens': 'jugement',
 'placités': 'pl cités',
 'changemens': 'changement',
 'assujéties': 'assuré les',
 'appelans': 'appelons',
 'instrumenlaires': 'instrumentaires',
 'rejelée': 'rejette',
 'originaiie': 'originaire',
 'demaine': 'domaine',
 'privilége': 'privilège'}

In [95]:
c.spell_check("melodrame et juje")

melodrame -> mélodrame
juge, 1, 9378326
juje -> juge


'melodrame et juje'